In [4]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
from renormalization import find_Jc, dH_dH, rg_step
from utils import required_initial_max_distance, build_J

In [5]:
def magnetization(J0, a, Jc, max_dist_final=3, max_steps=10, eps=1e-8):
    # If we start on the disordered side, magnetization is zero.
    M_sink = 1.0 if J0 > Jc else 0.0
    if M_sink == 0.0:
        return 0.0

    # Length-rescaling factor and spatial dimension
    b = 3.0
    d_dim = 1.0

    D0 = required_initial_max_distance(max_dist_final, max_steps)
    J = build_J(J0, a, D0)

    prod_R = 1.0
    steps_done = 0

    for step in range(max_steps):
        # dH'/dH at this level
        dH = dH_dH(J, eps=eps)
        if not np.isfinite(dH) or dH <= 0.0:
            break
        prod_R *= dH
        steps_done = step + 1

        # Next RG step. If the range collapses (r_max < 1) we stop.
        D = len(J) - 1
        r_max = (D - 2) // 3
        if r_max < 1:
            break
        J = rg_step(J)

    if steps_done == 0:
        return 0.0

    scale = (b ** (-d_dim)) ** steps_done
    M0 = scale * prod_R * M_sink
    return float(M0)

In [6]:
a = 1

Jc = find_Jc(a, Jlow=1e-2, Jhigh=1e2,
     max_steps=6, max_dist_final=9, tol=1e-5,
     growth_threshold=1e4, decay_threshold=1e-4)
print("Critical J =", Jc)

J = Jc + .001
M = magnetization(J, a, Jc, max_dist_final=9, max_steps=7)
print("M =", M)

Critical J = 0.010002979934215547
M = 0.013981746743412848


In [7]:
# Parameters
a = 1.5
Jc = find_Jc(a=a, Jlow=1e-2, Jhigh=1e2,
             max_steps=8, max_dist_final=6, tol=1e-6,
             growth_threshold=1e4, decay_threshold=1e-4)

print(f"Jc = {Jc}")
T_values = np.linspace(0.1, 1.5/Jc, 100)
J_values = 1.0 / T_values

M_values = [magnetization(J0, a, Jc,
                          max_dist_final=6,
                          max_steps=12,
                          eps=1e-8) for J0 in J_values]

plt.figure(figsize=(7, 5))
plt.plot(T_values, M_values, 'b-o')
plt.xlabel("Temperature T = 1/J")
plt.ylabel("Magnetization M")
plt.ylim(0, 1.05)
plt.grid(True)
plt.show()

Jc = 0.010000372491776943


KeyboardInterrupt: 

In [ ]:
from matplotlib import cm

a_values = np.array([0.0 , 0.2, 0.4, 0.6])
data_by_a = []

for a in a_values:
    print(f"Processing a = {a}")

    if a < 0.8:
        Jc = 1e-10
        T_arr = np.linspace(0.10, 100, 300)
        J_arr = 1.0 / T_arr
    else:
        Jc = find_Jc(
            a=a,
            Jlow=1e-2, Jhigh=1e2,
            max_steps=10,
            max_dist_final=6,
            tol=1e-6,
            growth_threshold=1e4,
            decay_threshold=1e-4
        )
        T_arr = np.linspace(0.10, 1.1 / Jc, 100)
        J_arr = 1.0 / T_arr


    M_arr = []
    for J0 in J_arr:
        T = 1/J0
        if T < 0.8: max_k = 8
        else: max_k = 10
        M_arr.append(magnetization(J0, a, Jc, max_dist_final=6, max_steps=max_k, eps=1e-8))    
    M_arr = np.array(M_arr)

    data_by_a.append(list(zip(T_arr, M_arr)))

# Plot
fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.copper
norm = plt.Normalize(a_values.min(), a_values.max())

for a, tuples_a in zip(a_values, data_by_a):
    T_arr = np.array([x[0] for x in tuples_a])
    M_arr = np.array([x[1] for x in tuples_a])
    ax.plot(T_arr, M_arr, label=f"{a:.3f}", color=cmap(norm(a)), linewidth=2)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
fig.colorbar(sm, ax=ax, label='Interaction range exponent a')
ax.set_xlabel('Temperature  1/J')
ax.set_ylabel('Magnetization  M')
ax.set_xlim(0, None)
ax.set_ylim(0, 1.05)
ax.legend(title='a', loc='upper right', fontsize=9, framealpha=.9)
ax.tick_params(axis='both', which='major', direction='in', width=1, length=6)
plt.tight_layout()
plt.show()

In [ ]:
from matplotlib import cm

a_values = np.array([0.0 , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 1.9, 2.0 ])
data_by_a = []

for a in a_values:
    print(f"Processing a = {a}")

    Jc = find_Jc(
        a=a,
        Jlow=1e-5, Jhigh=1e2,
        max_steps=10,
        max_dist_final=6,
        tol=1e-6,
        growth_threshold=1e4,
        decay_threshold=1e-4
    )

    T_arr = np.linspace(0.10, 1.5 / Jc, 300)
    J_arr = 1.0 / T_arr

    M_arr = np.array([
        magnetization(J0, a, Jc, max_dist_final=6, max_steps=10, eps=1e-8)
        for J0 in J_arr
    ])

    data_by_a.append(list(zip(T_arr, M_arr)))

# Plot
fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.copper
norm = plt.Normalize(a_values.min(), a_values.max())

for a, tuples_a in zip(a_values, data_by_a):
    T_arr = np.array([x[0] for x in tuples_a])
    M_arr = np.array([x[1] for x in tuples_a])
    ax.plot(T_arr, M_arr, label=f"{a:.3f}", color=cmap(norm(a)), linewidth=2)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
fig.colorbar(sm, ax=ax, label='Interaction range exponent a')
ax.set_xlabel('Temperature  1/J')
ax.set_ylabel('Magnetization  M')
ax.set_xlim(0, 100)
ax.set_ylim(0, 1.05)
ax.legend(title='a', loc='upper right', fontsize=9, framealpha=.9)
ax.tick_params(axis='both', which='major', direction='in', width=1, length=6)
plt.tight_layout()
plt.show()

In [ ]:
# Save data for each a to its own file
for a, tuples_a in zip(a_values, data_by_a):
    T_arr = np.array([x[0] for x in tuples_a])
    M_arr = np.array([x[1] for x in tuples_a])
    out = np.column_stack((T_arr, M_arr))

    filename = f"../data/magnetization/magnetization_a_{a:.3f}.txt"
    np.savetxt(
        filename, out,
        header="T\tM",
        fmt="%.12e",
        comments=""
    )
    print("Saved:", filename)

In [ ]:
# Load data for each a from its own file
import numpy as np

# a-values used originally
a_values = np.array([0.0 , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 1.9, 2.0])

data_by_a = []

for a in a_values:
    filename = f"../data/magnetization/magnetization_a_{a:.3f}.txt"
    print("Loading:", filename)

    # If file has a header "T M", skiprows=1 handles it.
    arr = np.loadtxt(filename, skiprows=1)

    T_arr = arr[:, 0]
    M_arr = arr[:, 1]

    data_by_a.append(list(zip(T_arr, M_arr)))

In [ ]:
# Plot loaded data
from matplotlib import cm

plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'mathtext.default': 'regular'})

fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.copper
norm = plt.Normalize(a_values.min(), a_values.max())

# Main plot
for a, tuples_a in zip(a_values, data_by_a):
    T_arr = np.array([x[0] for x in tuples_a])
    M_arr = np.array([x[1] for x in tuples_a])
    if a != max(a_values):
        ax.plot(T_arr, M_arr, label=f"{a:.1f}", color=cmap(norm(a)), linewidth=2)
    else:
        i = np.where(T_arr==1.054473426129e+00)[0][0]
        ax.plot(T_arr[:i+1], M_arr[:i+1], marker='', label=f'{a:.1f}', color=cmap(norm(a)), linewidth=1.5)
        ax.plot([T_arr[i], T_arr[i]], [0, M_arr[i]], linestyle='--', color=cmap(norm(a)), linewidth=1.5)


# Colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label('Interaction range exponent $a$', fontsize=16)
cbar.ax.tick_params(labelsize=14) 

ax.set_xlabel('Temperature  $1/J$', fontsize=16)
ax.set_ylabel('Magnetization  $M$', fontsize=16)
ax.set_xlim(0, 80)
ax.set_ylim(0, 1.05)
ax.legend(title='a', loc=(0.18, 0.5), fontsize=9, framealpha=.9)
ax.tick_params(axis='both', which='major', direction='in', labelsize=14, width=1, length=6)

# --- Inset axis (upper right) ---
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

inset = inset_axes(ax, width="60%", height="60%", loc="upper right")

# Plot into inset
for a, tuples_a in zip(a_values, data_by_a):
    T_arr = np.array([x[0] for x in tuples_a])
    M_arr = np.array([x[1] for x in tuples_a])
    if a != max(a_values):
        inset.plot(T_arr, M_arr, color=cmap(norm(a)), linewidth=1.5)
    else:
        #i = np.where(T_arr==1.054473426129e+00)[0][0]
        inset.plot(T_arr[:i+1], M_arr[:i+1], marker='', label=f'{a:.2f}', color=cmap(norm(a)), linewidth=1.5)
        inset.plot([T_arr[i], T_arr[i]], [0, M_arr[i]], linestyle='--', color=cmap(norm(a)), linewidth=1.5)

# Set x-axis limits for zoom
inset.set_xlabel('$1/J$', fontsize=16)
inset.set_ylabel('$M$', fontsize=16)
inset.set_xlim(0, 3.2)
inset.set_ylim(0, 1.05)
inset.tick_params(axis='both', which='major', direction='in', labelsize=14, width=1, length=6)

plt.tight_layout()
plt.savefig("magnetization.png")
plt.show()